In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
holiday_data = pd.read_csv('../raw_data/holidays_events.csv')

In [4]:
train_data = pd.read_csv('../raw_data/train2016.csv')
train_data.drop(columns='Unnamed: 0', inplace=True)

In [5]:
# train_itemsp = pd.read_csv('../raw_data/train2016_perish.csv')
# train_itemsp.drop(columns='Unnamed: 0', inplace=True)

In [6]:
# oil_data = pd.read_csv('../raw_data/oil.csv')

stores_data = pd.read_csv('../raw_data/stores.csv')

items_data = pd.read_csv('../raw_data/items.csv')

# transactions_data = pd.read_csv('../raw_data/transactions.csv')

# Additional replace

Remplacer les Additional par le type d'événement concerné. Il n'y a que 51 lignes concernées, ça peut se faire à la main directement sur le CSV de base

In [7]:
# check day of week
holiday_data['date'] = pd.to_datetime(holiday_data['date'])
holiday_data.date.dt.dayofweek.unique() # 0 = monday ; 6 = sunday

array([4, 6, 3, 5, 0, 1, 2])

## Convert Additional

In [8]:
# Additional to interpret same type everywhere

In [9]:
holiday_data[holiday_data['type'] == 'Additional']

date        type    locale locale_name               description  \
28  2012-12-05  Additional     Local       Quito      Fundacion de Quito-1   
31  2012-12-21  Additional  National     Ecuador                 Navidad-4   
33  2012-12-22  Additional  National     Ecuador                 Navidad-3   
34  2012-12-23  Additional  National     Ecuador                 Navidad-2   
36  2012-12-24  Additional  National     Ecuador                 Navidad-1   
38  2012-12-26  Additional  National     Ecuador                 Navidad+1   
40  2012-12-31  Additional  National     Ecuador      Primer dia del ano-1   
53  2013-05-11  Additional  National     Ecuador         Dia de la Madre-1   
64  2013-07-24  Additional     Local   Guayaquil  Fundacion de Guayaquil-1   
81  2013-12-05  Additional     Local       Quito      Fundacion de Quito-1   
84  2013-12-21  Additional  National     Ecuador                 Navidad-4   
85  2013-12-22  Additional  National     Ecuador                 Navidad-3   
87  2013-12-23  Additional  National     Ecuador                 Navidad-2   
88  2013-12-24  Additional  National     Ecuador                 Navidad-1   
90  2013-12-26  Additional  National     Ecuador                 Navidad+1   
91  2013-12-31  Additional  National     Ecuador      Primer dia del ano-1   
102 2014-05-10  Additional  National     Ecuador         Dia de la Madre-1   
127 2014-07-24  Additional     Local   Guayaquil  Fundacion de Guayaquil-1   
146 2014-12-05  Additional     Local       Quito      Fundacion de Quito-1   
150 2014-12-21  Additional  National     Ecuador                 Navidad-4   
152 2014-12-22  Additional  National     Ecuador                 Navidad-3   
153 2014-12-23  Additional  National     Ecuador                 Navidad-2   
154 2014-12-24  Additional  National     Ecuador                 Navidad-1   
157 2014-12-26  Additional  National     Ecuador                 Navidad+1   
158 2014-12-31  Additional  National     Ecuador      Primer dia del ano-1   
171 2015-05-09  Additional  National     Ecuador         Dia de la Madre-1   
200 2015-12-05  Additional     Local       Quito      Fundacion de Quito-1   
203 2015-12-21  Additional  National     Ecuador                 Navidad-4   
204 2015-12-22  Additional  National     Ecuador                 Navidad-3   
206 2015-12-23  Additional  National     Ecuador                 Navidad-2   
207 2015-12-24  Additional  National     Ecuador                 Navidad-1   
209 2015-12-26  Additional  National     Ecuador                 Navidad+1   
210 2015-12-31  Additional  National     Ecuador      Primer dia del ano-1   
242 2016-05-07  Additional  National     Ecuador         Dia de la Madre-1   
264 2016-07-24  Additional     Local   Guayaquil  Fundacion de Guayaquil-1   
286 2016-12-05  Additional     Local       Quito      Fundacion de Quito-1   
289 2016-12-21  Additional  National     Ecuador                 Navidad-4   
290 2016-12-22  Additional  National     Ecuador                 Navidad-3   
292 2016-12-23  Additional  National     Ecuador                 Navidad-2   
293 2016-12-24  Additional  National     Ecuador                 Navidad-1   
295 2016-12-26  Additional  National     Ecuador                 Navidad+1   
296 2016-12-31  Additional  National     Ecuador      Primer dia del ano-1   
310 2017-05-13  Additional  National     Ecuador         Dia de la Madre-1   
321 2017-07-24  Additional     Local   Guayaquil  Fundacion de Guayaquil-1   
322 2017-07-25  Additional     Local   Guayaquil    Fundacion de Guayaquil   
339 2017-12-05  Additional     Local       Quito      Fundacion de Quito-1   
343 2017-12-21  Additional  National     Ecuador                 Navidad-4   
345 2017-12-22  Additional  National     Ecuador                 Navidad-3   
346 2017-12-23  Additional  National     Ecuador                 Navidad-2   
347 2017-12-24  Additional  National     Ecuador                 Navidad-1   
349 2017-12-26  Additional  Nati

In [10]:
holiday_data[holiday_data['type'] == 'Additional'].shape

(51, 6)

In [11]:
# obtain 'Fundacion de Quito' with slider in Additional in 1 row in description
holiday_data[holiday_data['type'] == 'Additional'].loc[322]['description']

'Fundacion de Guayaquil'

**Review loop (Because, except interrupt loop)**

In [12]:
# copy holiday_additional, more safe maybe...

while len(holiday_data[holiday_data['type'] == 'Additional']) != 0:
    
    holiday_additional = holiday_data[holiday_data['type'] == 'Additional']
    for index, one_additional in holiday_additional.iterrows():
        
        # check description
        description = one_additional['description']
        # check if convert is possible (Sometimes, description value isn't completed)
        try:
            counter = int(one_additional['description'][-1])

            # if '+' or '-'
            if description[-2] == '+':
                # check previous and changed additional type with it
                holiday_data['type'][index] = holiday_data.loc[index - counter]['type']

            elif description[-2] == '-':
                # check next true holiday and same
                holiday_data['type'][index] = holiday_data.loc[index + counter]['type']

        except:
            # sometimes, not - and + ... Then assign directly first type founded
            replace_default = description[:-2]
            holiday_data['type'][index] = holiday_data[holiday_data['description'] == description]['type'].iloc[0]

/tmp/ipykernel_622/3528670214.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holiday_data['type'][index] = holiday_data.loc[index + counter]['type']
/tmp/ipykernel_622/3528670214.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holiday_data['type'][index] = holiday_data.loc[index - counter]['type']
/tmp/ipykernel_622/3528670214.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holiday_data['type'][index] = holiday_data[hol

In [13]:
# lauch loop twice, i don't know why...
holiday_data.loc[holiday_data['type'] == 'Additional'] # not found normally

Empty DataFrame
Columns: [date, type, locale, locale_name, description, transferred]
Index: []

In [14]:
# description with -X or +X, type changed
holiday_data[holiday_data['description'] == 'Navidad-1']

date     type    locale locale_name description  transferred
36  2012-12-24  Holiday  National     Ecuador   Navidad-1        False
88  2013-12-24  Holiday  National     Ecuador   Navidad-1        False
154 2014-12-24  Holiday  National     Ecuador   Navidad-1        False
207 2015-12-24  Holiday  National     Ecuador   Navidad-1        False
293 2016-12-24  Holiday  National     Ecuador   Navidad-1        False
347 2017-12-24  Holiday  National     Ecuador   Navidad-1        False

In [15]:
# file.csv save
# version 2 of holiday_events
holiday_data.to_csv('../raw_data/holidays_events_v2.csv')

# Generate df_base

Générer un dataframe df_base avec 3 colonnes : date, store_id et item_id qui contient toutes les dates (entre la première et la dernière date de vos données) et, pour chacune de ces dates, tous les stores existants et tous les items

**Seulement sur 2016**

In [16]:
min(train_data['date']), max(train_data['date'])

('2016-01-01', '2016-12-31')

In [17]:
train_data['store_nbr'].unique()

array([25,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       53, 54])

In [18]:
rng = pd.date_range(start='2016-01-01', end='2016-03-31')
store_nbr_series = train_data['store_nbr'].unique()

In [19]:
df_all_store = pd.DataFrame({'store_nbr': store_nbr_series})
df_all_store

store_nbr
0          25
1           1
2           2
3           3
4           4
5           5
6           6
7           7
8           8
9           9
10         10
11         11
12         12
13         13
14         14
15         15
16         16
17         17
18         18
19         19
20         20
21         21
22         22
23         23
24         24
25         26
26         27
27         28
28         29
29         30
30         31
31         32
32         33
33         34
34         35
35         36
36         37
37         38
38         39
39         40
40         41
41         42
42         43
43         44
44         45
45         46
46         47
47         48
48         49
49         50
50         51
51         53
52         54

In [20]:
df_base = pd.DataFrame({'date': rng})
df_base

date
0  2016-01-01
1  2016-01-02
2  2016-01-03
3  2016-01-04
4  2016-01-05
..        ...
86 2016-03-27
87 2016-03-28
88 2016-03-29
89 2016-03-30
90 2016-03-31

[91 rows x 1 columns]

In [21]:
df_base = df_base.merge(df_all_store, how='cross')
df_base

date  store_nbr
0    2016-01-01         25
1    2016-01-01          1
2    2016-01-01          2
3    2016-01-01          3
4    2016-01-01          4
...         ...        ...
4818 2016-03-31         49
4819 2016-03-31         50
4820 2016-03-31         51
4821 2016-03-31         53
4822 2016-03-31         54

[4823 rows x 2 columns]

In [22]:
item_nbr_series = train_data['item_nbr'].unique()
item_nbr_series

array([ 105574,  105575,  105857, ..., 2028217, 2042947, 2042314])

In [23]:
len(item_nbr_series)

3886

In [24]:
df_item = pd.DataFrame({'item_nbr': item_nbr_series})
df_item

item_nbr
0       105574
1       105575
2       105857
3       108634
4       108701
...        ...
3881   2042941
3882   2043198
3883   2028217
3884   2042947
3885   2042314

[3886 rows x 1 columns]

In [25]:
df_base = df_base.merge(df_item, how='cross')
df_base

date  store_nbr  item_nbr
0        2016-01-01         25    105574
1        2016-01-01         25    105575
2        2016-01-01         25    105857
3        2016-01-01         25    108634
4        2016-01-01         25    108701
...             ...        ...       ...
18742173 2016-03-31         54   2042941
18742174 2016-03-31         54   2043198
18742175 2016-03-31         54   2028217
18742176 2016-03-31         54   2042947
18742177 2016-03-31         54   2042314

[18742178 rows x 3 columns]

In [26]:
del df_all_store, df_item

# First `df_sales`

À partir du df_base, faire un left_join sur date, store_id, item_id avec train, pour récupérer les unit_sales et remplacer les Nan par des 0 :flèche_droite: df_sales

In [27]:
train_data.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
       'year'],
      dtype='object')

In [28]:
df_base['date'] = pd.to_datetime(df_base['date'])

In [29]:
train_data['date'] = pd.to_datetime(train_data['date'])

In [30]:
df_sales = df_base.merge(train_data, how='left', on=['date', 'store_nbr', 'item_nbr'])
df_sales

date  store_nbr  item_nbr          id  unit_sales onpromotion  \
0        2016-01-01         25    105574  66458908.0        12.0       False   
1        2016-01-01         25    105575  66458909.0         9.0       False   
2        2016-01-01         25    105857  66458910.0         3.0       False   
3        2016-01-01         25    108634  66458911.0         3.0       False   
4        2016-01-01         25    108701  66458912.0         2.0        True   
...             ...        ...       ...         ...         ...         ...   
18742173 2016-03-31         54   2042941         NaN         NaN         NaN   
18742174 2016-03-31         54   2043198         NaN         NaN         NaN   
18742175 2016-03-31         54   2028217         NaN         NaN         NaN   
18742176 2016-03-31         54   2042947         NaN         NaN         NaN   
18742177 2016-03-31         54   2042314         NaN         NaN         NaN   

            year  
0         2016.0  
1         2016.0  
2         2016.0  
3         2016.0  
4         2016.0  
...          ...  
18742173     NaN  
18742174     NaN  
18742175     NaN  
18742176     NaN  
18742177     NaN  

[18742178 rows x 7 columns]

In [31]:
df_sales['unit_sales'] = df_sales['unit_sales'].fillna(0)
df_sales[df_sales['unit_sales'] == 0]

date  store_nbr  item_nbr  id  unit_sales onpromotion  year
1718     2016-01-01         25    103520 NaN         0.0         NaN   NaN
1719     2016-01-01         25    105577 NaN         0.0         NaN   NaN
1720     2016-01-01         25    106716 NaN         0.0         NaN   NaN
1721     2016-01-01         25    108079 NaN         0.0         NaN   NaN
1722     2016-01-01         25    108696 NaN         0.0         NaN   NaN
...             ...        ...       ...  ..         ...         ...   ...
18742173 2016-03-31         54   2042941 NaN         0.0         NaN   NaN
18742174 2016-03-31         54   2043198 NaN         0.0         NaN   NaN
18742175 2016-03-31         54   2028217 NaN         0.0         NaN   NaN
18742176 2016-03-31         54   2042947 NaN         0.0         NaN   NaN
18742177 2016-03-31         54   2042314 NaN         0.0         NaN   NaN

[10238488 rows x 7 columns]

# Groupby

On considère un magasin fermé s'il n'a aucune vente ce jour là. Il faut donc créer un dataframe à partir du précédent, avec un groupby par exemple (somme sur les unit_sales), pour obtenir le nombre total de ventes par magasin par jour. Sur ce nouveau dataframe, créer une colonne is_open :flèche_droite: df_open

In [32]:
df_open = df_sales[['date', 'store_nbr', 'unit_sales']].groupby(by=['date', 'store_nbr']).sum()

In [33]:
df_open.reset_index(inplace=True)

In [34]:
df_open

date  store_nbr  unit_sales
0    2016-01-01          1       0.000
1    2016-01-01          2       0.000
2    2016-01-01          3       0.000
3    2016-01-01          4       0.000
4    2016-01-01          5       0.000
...         ...        ...         ...
4818 2016-03-31         49   27019.048
4819 2016-03-31         50   14005.689
4820 2016-03-31         51   15628.633
4821 2016-03-31         53    5923.787
4822 2016-03-31         54    5168.700

[4823 rows x 3 columns]

In [35]:
df_open['is_open'] = (df_open['unit_sales'] != 0)
df_open

date  store_nbr  unit_sales  is_open
0    2016-01-01          1       0.000    False
1    2016-01-01          2       0.000    False
2    2016-01-01          3       0.000    False
3    2016-01-01          4       0.000    False
4    2016-01-01          5       0.000    False
...         ...        ...         ...      ...
4818 2016-03-31         49   27019.048     True
4819 2016-03-31         50   14005.689     True
4820 2016-03-31         51   15628.633     True
4821 2016-03-31         53    5923.787     True
4822 2016-03-31         54    5168.700     True

[4823 rows x 4 columns]

# join df_sales df_open

Faire un left join entre df_sales et df_open sur les colonnes de dates et de store_id :flèche_droite: df_sales

In [36]:
df_sales = df_sales.merge(df_open[['date', 'store_nbr', 'is_open']], how='left', on=['date', 'store_nbr'])
df_sales

date  store_nbr  item_nbr          id  unit_sales onpromotion  \
0        2016-01-01         25    105574  66458908.0        12.0       False   
1        2016-01-01         25    105575  66458909.0         9.0       False   
2        2016-01-01         25    105857  66458910.0         3.0       False   
3        2016-01-01         25    108634  66458911.0         3.0       False   
4        2016-01-01         25    108701  66458912.0         2.0        True   
...             ...        ...       ...         ...         ...         ...   
18742173 2016-03-31         54   2042941         NaN         0.0         NaN   
18742174 2016-03-31         54   2043198         NaN         0.0         NaN   
18742175 2016-03-31         54   2028217         NaN         0.0         NaN   
18742176 2016-03-31         54   2042947         NaN         0.0         NaN   
18742177 2016-03-31         54   2042314         NaN         0.0         NaN   

            year  is_open  
0         2016.0     True  
1         2016.0     True  
2         2016.0     True  
3         2016.0     True  
4         2016.0     True  
...          ...      ...  
18742173     NaN     True  
18742174     NaN     True  
18742175     NaN     True  
18742176     NaN     True  
18742177     NaN     True  

[18742178 rows x 8 columns]

# df_holiday['is_special']

Préparer le df_holiday : ajouter is_special et la city à partir des Local, Regional, National

In [37]:
holiday_data.drop(columns=['description', 'transferred'], inplace=True)

## Create city_state woth stores

In [38]:
city_state = stores_data[['city', 'state']].drop_duplicates()
city_state.head()

city                           state
0           Quito                       Pichincha
4   Santo Domingo  Santo Domingo de los Tsachilas
10        Cayambe                       Pichincha
11      Latacunga                        Cotopaxi
13       Riobamba                      Chimborazo

## Prepare local

In [39]:
local_holiday = holiday_data[holiday_data.locale == 'Local']

# new column city in local_holiday, useful for merging
local_holiday['city'] = local_holiday['locale_name']
local_holiday.head()

/tmp/ipykernel_622/2905539151.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  local_holiday['city'] = local_holiday['locale_name']


date     type locale locale_name      city
0 2012-03-02  Holiday  Local       Manta     Manta
2 2012-04-12  Holiday  Local      Cuenca    Cuenca
3 2012-04-14  Holiday  Local    Libertad  Libertad
4 2012-04-21  Holiday  Local    Riobamba  Riobamba
5 2012-05-12  Holiday  Local        Puyo      Puyo

## Prepare regional only

In [40]:
regional_holiday = holiday_data.loc[holiday_data.locale == 'Regional']

In [41]:
regional_holiday = regional_holiday.merge(city_state, left_on='locale_name', right_on='state')
regional_holiday.head()

date     type    locale locale_name       city     state
0 2012-04-01  Holiday  Regional    Cotopaxi  Latacunga  Cotopaxi
1 2013-04-01  Holiday  Regional    Cotopaxi  Latacunga  Cotopaxi
2 2014-04-01  Holiday  Regional    Cotopaxi  Latacunga  Cotopaxi
3 2015-04-01  Holiday  Regional    Cotopaxi  Latacunga  Cotopaxi
4 2016-04-01  Holiday  Regional    Cotopaxi  Latacunga  Cotopaxi

In [42]:
# state not useful for merging
regional_holiday.drop(columns='state', inplace=True)

In [43]:
regional_holiday.head()

date     type    locale locale_name       city
0 2012-04-01  Holiday  Regional    Cotopaxi  Latacunga
1 2013-04-01  Holiday  Regional    Cotopaxi  Latacunga
2 2014-04-01  Holiday  Regional    Cotopaxi  Latacunga
3 2015-04-01  Holiday  Regional    Cotopaxi  Latacunga
4 2016-04-01  Holiday  Regional    Cotopaxi  Latacunga

## Prepare National only

In [44]:
national_holiday = holiday_data.loc[holiday_data.locale == 'National']
national_holiday.head()

date      type    locale locale_name
14 2012-08-10   Holiday  National     Ecuador
19 2012-10-09   Holiday  National     Ecuador
20 2012-10-12  Transfer  National     Ecuador
21 2012-11-02   Holiday  National     Ecuador
22 2012-11-03   Holiday  National     Ecuador

In [45]:
city_state.head()

city                           state
0           Quito                       Pichincha
4   Santo Domingo  Santo Domingo de los Tsachilas
10        Cayambe                       Pichincha
11      Latacunga                        Cotopaxi
13       Riobamba                      Chimborazo

In [46]:
# Add column country in city_state to merge easily after
city_state['country'] = 'Ecuador'
city_state.head()

city                           state  country
0           Quito                       Pichincha  Ecuador
4   Santo Domingo  Santo Domingo de los Tsachilas  Ecuador
10        Cayambe                       Pichincha  Ecuador
11      Latacunga                        Cotopaxi  Ecuador
13       Riobamba                      Chimborazo  Ecuador

In [47]:
national_holiday = national_holiday.merge(city_state, left_on='locale_name', right_on='country')
national_holiday.head()

date     type    locale locale_name           city  \
0 2012-08-10  Holiday  National     Ecuador          Quito   
1 2012-08-10  Holiday  National     Ecuador  Santo Domingo   
2 2012-08-10  Holiday  National     Ecuador        Cayambe   
3 2012-08-10  Holiday  National     Ecuador      Latacunga   
4 2012-08-10  Holiday  National     Ecuador       Riobamba   

                            state  country  
0                       Pichincha  Ecuador  
1  Santo Domingo de los Tsachilas  Ecuador  
2                       Pichincha  Ecuador  
3                        Cotopaxi  Ecuador  
4                      Chimborazo  Ecuador

In [48]:
national_holiday.drop(columns=['state', 'country'], inplace=True)

## `pd.concat` each `locale` with 3 previous variables

In [49]:
# local_holiday
# regional_holiday
# national_holiday

df_holiday = pd.concat([local_holiday, regional_holiday, national_holiday])[['date', 'type', 'city']].drop_duplicates()
df_holiday['is_special'] = True
df_holiday.head()

date     type      city  is_special
0 2012-03-02  Holiday     Manta        True
2 2012-04-12  Holiday    Cuenca        True
3 2012-04-14  Holiday  Libertad        True
4 2012-04-21  Holiday  Riobamba        True
5 2012-05-12  Holiday      Puyo        True

# Merge df_sales et stores

Merger df_sales avec les store pour récupérer l'information des city :flèche_droite: df_sales

In [50]:
df_sales.columns

Index(['date', 'store_nbr', 'item_nbr', 'id', 'unit_sales', 'onpromotion',
       'year', 'is_open'],
      dtype='object')

In [51]:
stores_data.columns

Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object')

In [52]:
df_sales = df_sales.merge(stores_data, how='left', on='store_nbr')

In [53]:
df_sales.head()

date  store_nbr  item_nbr          id  unit_sales onpromotion    year  \
0 2016-01-01         25    105574  66458908.0        12.0       False  2016.0   
1 2016-01-01         25    105575  66458909.0         9.0       False  2016.0   
2 2016-01-01         25    105857  66458910.0         3.0       False  2016.0   
3 2016-01-01         25    108634  66458911.0         3.0       False  2016.0   
4 2016-01-01         25    108701  66458912.0         2.0        True  2016.0   

   is_open     city        state type  cluster  
0     True  Salinas  Santa Elena    D        1  
1     True  Salinas  Santa Elena    D        1  
2     True  Salinas  Santa Elena    D        1  
3     True  Salinas  Santa Elena    D        1  
4     True  Salinas  Santa Elena    D        1

# Merge df_sales et df_holiday

Merger df_sales avec df_holdiay sur date et city :flèche_droite: df_sales

In [54]:
df_holiday.head()

date     type      city  is_special
0 2012-03-02  Holiday     Manta        True
2 2012-04-12  Holiday    Cuenca        True
3 2012-04-14  Holiday  Libertad        True
4 2012-04-21  Holiday  Riobamba        True
5 2012-05-12  Holiday      Puyo        True

In [55]:
df_holiday['date'] = pd.to_datetime(df_holiday['date'])

In [56]:
df_sales = df_sales.merge(df_holiday, how='left', on=['date', 'city'])

In [57]:
df_sales.head()

date  store_nbr  item_nbr          id  unit_sales onpromotion    year  \
0 2016-01-01         25    105574  66458908.0        12.0       False  2016.0   
1 2016-01-01         25    105575  66458909.0         9.0       False  2016.0   
2 2016-01-01         25    105857  66458910.0         3.0       False  2016.0   
3 2016-01-01         25    108634  66458911.0         3.0       False  2016.0   
4 2016-01-01         25    108701  66458912.0         2.0        True  2016.0   

   is_open     city        state type_x  cluster   type_y is_special  
0     True  Salinas  Santa Elena      D        1  Holiday       True  
1     True  Salinas  Santa Elena      D        1  Holiday       True  
2     True  Salinas  Santa Elena      D        1  Holiday       True  
3     True  Salinas  Santa Elena      D        1  Holiday       True  
4     True  Salinas  Santa Elena      D        1  Holiday       True

In [58]:
df_sales['is_special'].fillna(False, inplace=True)

# Merge df_sales et items

Merger df_sales avec items 

In [59]:
items_data.columns

Index(['item_nbr', 'family', 'class', 'perishable'], dtype='object')

In [60]:
df_sales = df_sales.merge(items_data, how='left', on='item_nbr')
df_sales.head()

date  store_nbr  item_nbr          id  unit_sales onpromotion    year  \
0 2016-01-01         25    105574  66458908.0        12.0       False  2016.0   
1 2016-01-01         25    105575  66458909.0         9.0       False  2016.0   
2 2016-01-01         25    105857  66458910.0         3.0       False  2016.0   
3 2016-01-01         25    108634  66458911.0         3.0       False  2016.0   
4 2016-01-01         25    108701  66458912.0         2.0        True  2016.0   

   is_open     city        state type_x  cluster   type_y  is_special  \
0     True  Salinas  Santa Elena      D        1  Holiday        True   
1     True  Salinas  Santa Elena      D        1  Holiday        True   
2     True  Salinas  Santa Elena      D        1  Holiday        True   
3     True  Salinas  Santa Elena      D        1  Holiday        True   
4     True  Salinas  Santa Elena      D        1  Holiday        True   

      family  class  perishable  
0  GROCERY I   1045           0  
1  GROCERY I   1045           0  
2  GROCERY I   1092           0  
3  GROCERY I   1075           0  
4       DELI   2644           1

# Generate columns year month day dayofweek

In [61]:
df_sales.drop(column='year')

TypeError: drop() got an unexpected keyword argument 'column'

In [ ]:
df_sales['year'] = df_sales['date'].dt.year

In [ ]:
df_salesles['month'] = df_sales['date'].dt.month

In [ ]:
df_sales['day'] = df_sales['date'].dt.day

In [ ]:
df_sales['dayofweek'] = df_sales['date'].dt.dayofweek

In [ ]:
df_sales.to_csv('../raw_data/train_all_table.csv')

In [64]:
import math

In [66]:
df_sales['item_nbr'].isna().unique()

array([False])

In [67]:
df_sales.head(1000)

date  store_nbr  item_nbr          id  unit_sales onpromotion  \
0   2016-01-01         25    105574  66458908.0        12.0       False   
1   2016-01-01         25    105575  66458909.0         9.0       False   
2   2016-01-01         25    105857  66458910.0         3.0       False   
3   2016-01-01         25    108634  66458911.0         3.0       False   
4   2016-01-01         25    108701  66458912.0         2.0        True   
..         ...        ...       ...         ...         ...         ...   
995 2016-01-01         25   1229023  66459903.0         3.0       False   
996 2016-01-01         25   1229025  66459904.0         7.0       False   
997 2016-01-01         25   1229028  66459905.0         5.0       False   
998 2016-01-01         25   1229469  66459906.0         2.0       False   
999 2016-01-01         25   1229500  66459907.0         1.0       False   

       year  is_open     city        state type_x  cluster   type_y  \
0    2016.0     True  Salinas  Santa Elena      D        1  Holiday   
1    2016.0     True  Salinas  Santa Elena      D        1  Holiday   
2    2016.0     True  Salinas  Santa Elena      D        1  Holiday   
3    2016.0     True  Salinas  Santa Elena      D        1  Holiday   
4    2016.0     True  Salinas  Santa Elena      D        1  Holiday   
..      ...      ...      ...          ...    ...      ...      ...   
995  2016.0     True  Salinas  Santa Elena      D        1  Holiday   
996  2016.0     True  Salinas  Santa Elena      D        1  Holiday   
997  2016.0     True  Salinas  Santa Elena      D        1  Holiday   
998  2016.0     True  Salinas  Santa Elena      D        1  Holiday   
999  2016.0     True  Salinas  Santa Elena      D        1  Holiday   

     is_special         family  class  perishable  
0          True      GROCERY I   1045           0  
1          True      GROCERY I   1045           0  
2          True      GROCERY I   1092           0  
3          True      GROCERY I   1075           0  
4          True           DELI   2644           1  
..          ...            ...    ...         ...  
995        True  PERSONAL CARE   4176           0  
996        True  PERSONAL CARE   4176           0  
997        True  PERSONAL CARE   4176           0  
998        True  PERSONAL CARE   4138           0  
999        True  PERSONAL CARE   4162           0  

[1000 rows x 17 columns]